<a href="https://colab.research.google.com/github/sulthonarifimadudin/uasDL/blob/main/SQuAD/SQuAD_(Question_Answering)_ch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install -U transformers datasets accelerate evaluate sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 12.5 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

ds = load_dataset("rajpurkar/squad")  # train, validation
ds


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
print(ds["train"].column_names)
print(ds["train"][0])


['id', 'title', 'context', 'question', 'answers']
{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}


In [ ]:
from transformers import AutoTokenizer

model_ckpt = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

max_input_length  = 512
max_target_length = 64

def preprocess(batch):
    inputs = ["question: " + q.strip() + "  context: " + c.strip()
              for q, c in zip(batch["question"], batch["context"])]

    # ambil jawaban pertama sebagai target
    targets = [ans["text"][0].strip() if len(ans["text"]) > 0 else ""
               for ans in batch["answers"]]

    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True
    )

    labels = tokenizer(
        targets,
        max_length=max_target_length,
        truncation=True
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized = ds.map(preprocess, batched=True, remove_columns=ds["train"].column_names)
tokenized


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10570
    })
})

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
import numpy as np
import evaluate

squad_metric = evaluate.load("squad")

def postprocess_text(s):
    return " ".join(s.strip().split())

def compute_metrics(eval_pred):
    preds, labels = eval_pred

    # preds dari generate: token ids
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # labels berisi -100 untuk padding -> ganti pad_token_id biar bisa decode
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds  = [postprocess_text(p) for p in decoded_preds]
    decoded_labels = [postprocess_text(l) for l in decoded_labels]

    # id diambil dari dataset validation asli (urutan tokenized["validation"] sama)
    ids = ds["validation"]["id"]

    predictions = [{"id": i, "prediction_text": p} for i, p in zip(ids, decoded_preds)]
    references  = [{"id": i, "answers": a} for i, a in zip(ids, ds["validation"]["answers"])]

    return squad_metric.compute(predictions=predictions, references=references)


In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

args = Seq2SeqTrainingArguments(
    output_dir="t5-squad",
    eval_strategy="epoch", # Changed from evaluation_strategy
    save_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=1,              # naikin ke 2-3 kalau waktu cukup
    weight_decay=0.01,
    predict_with_generate=True,
    generation_max_length=max_target_length,
    fp16=True,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

/tmp/ipython-input-1371522385.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:wandb: You chose "Don't visualize my results"


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


OverflowError: out of range integral type conversion attempted

In [ ]:
import torch

def answer_question(question, context):
    text = f"question: {question}  context: {context}"
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_input_length).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=64,
            num_beams=4
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

sample = ds["validation"][0]
print("Q:", sample["question"])
print("A_pred:", answer_question(sample["question"], sample["context"]))
print("A_true:", sample["answers"]["text"][0])


Q: Which NFL team represented the AFC at Super Bowl 50?
A_pred: Denver Broncos
A_true: Denver Broncos
